In [1]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [2]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
allWords = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            allWords.append(temp1)
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [3]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [4]:
x = baseTrain
y = labels

In [5]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TFIDF = vectorizer_TF.fit_transform(x_min)
#X_TestTFIDF = vectorizer_TF.transform(baseTest)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer()
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [36]:
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
#clf = svm.SVR()
#clf = linear_model.LinearRegression()
#clf = DecisionTreeRegressor(max_depth=10)
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
#clf = GaussianNB()

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

#Classificação com probabilidade
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)

clf.fit(X_train,y_train_class)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
from sklearn import metrics
y_pred = clf.predict(X_test) 
y_pred_prob = clf.predict_proba(X_test) 
print(y_pred_prob)
print(X_test.shape)

y_pred_class = y_pred
y_test_class = []
y_pred_prob_tox = []


for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
    y_pred_prob_tox.append(y_pred_prob[t][0])

mse = metrics.mean_squared_error(y_test, y_pred_prob_tox)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

[[0.73336084 0.26663916]
 [0.87789703 0.12210297]
 [0.948108   0.051892  ]
 ...
 [0.72889449 0.27110551]
 [0.62911357 0.37088643]
 [0.89989449 0.10010551]]
(54011, 109767)
0.549067818797223
0.758345522208439
0.8748524203069658
0.10347716799329702
0.2806293159400008


In [ ]:
# from sklearn import metrics
# y_pred = clf.predict(X_test) 

# y_pred_class = []
# y_test_class = []

# for t in range(len(y_test)):
#     if(y_test[t]<0.5):
#         y_test_class.append(0)
#     else:
#         y_test_class.append(1)
#     if(y_pred[t]<0.5):
#         y_pred_class.append(0)
#     else:
#         y_pred_class.append(1)

# mse = metrics.mean_squared_error(y_test, y_pred)
# auc = metrics.roc_auc_score(y_test_class, y_pred_class)
# acc = metrics.accuracy_score(y_test_class, y_pred_class)
# prec = metrics.precision_score(y_test_class, y_pred_class)
# rec = metrics.recall_score(y_test_class, y_pred_class)

# print(auc)
# print(mse)
# print(acc)
# print(prec)
# print(rec)

# with open('results_validation.csv', mode='w') as employee_file:
#     writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["Linear Regression"])
#     writer.writerow(["mse: ", mse])
#     writer.writerow(["auc: ", auc])
#     writer.writerow(["accuracy: ", acc])
#     writer.writerow(["precision: ", prec])
#     writer.writerow(["recall: ", rec])

In [38]:
y_pred_test = clf.predict_proba(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id][1])
        writer.writerow([ids[id], y_pred_test[id][1]])

7000000
0.1537100558538564
7000001
0.14623520075568616
7000002
0.12871179772690056
7000003
0.04948255739632715
7000004
0.4978508953630421
7000005
0.11045852781075824
7000006
0.16855974576586583
7000007
0.15448651319681814
7000008
0.0904664540838602
7000009
0.09080126482505123
7000010
0.1924430722361837
7000011
0.24867450919657644
7000012
0.1262961993025444
7000013
0.0814579129359297
7000014
0.08664353916431411
7000015
0.07711308501784565
7000016
0.13168870194115345
7000017
0.06612725413834804
7000018
0.08268028238212329
7000019
0.15014868523855276
7000020
0.3332659800364067
7000021
0.1020222139897367
7000022
0.1231402585020772
7000023
0.2748245261248739
7000024
0.29502099714204616
7000025
0.15675384716610014
7000026
0.17706287089746467
7000027
0.08326586729889099
7000028
0.2081209865319576
7000029
0.04798604154166114
7000030
0.11329412062519273
7000031
0.08069689364869265
7000032
0.29013801360055447
7000033
0.08943018866695444
7000034
0.22805296187220717
7000035
0.14131462000486245
700

0.15274332481415245
7001024
0.17561131749463768
7001025
0.15178024265140536
7001026
0.04501969283350775
7001027
0.042436484308980706
7001028
0.12236767913876874
7001029
0.3440230771413992
7001030
0.10885724704340723
7001031
0.07397601107935066
7001032
0.08515089160623295
7001033
0.08249418721822838
7001034
0.16940752876792642
7001035
0.24731565278381493
7001036
0.13271722788139734
7001037
0.21113875446248936
7001038
0.03944581711223099
7001039
0.06776196726227103
7001040
0.1956494869397926
7001041
0.18055060762307037
7001042
0.13103876267792658
7001043
0.15296756547718826
7001044
0.18538427296765758
7001045
0.22784858594197843
7001046
0.11303520151614402
7001047
0.14356028796601827
7001048
0.2779948752129819
7001049
0.21722148821382692
7001050
0.20901442023414568
7001051
0.1268477011836166
7001052
0.16541531020393424
7001053
0.02977923707422044
7001054
0.23185821539005452
7001055
0.21040215983524505
7001056
0.17067826650512508
7001057
0.14927856903246142
7001058
0.056418574825194204
70

0.15362671466255198
7001850
0.2473771406459096
7001851
0.31769605060795847
7001852
0.14566523710471538
7001853
0.14006399687684712
7001854
0.17664075762779466
7001855
0.23151889242067294
7001856
0.02984308405744024
7001857
0.18362328537493333
7001858
0.19531803875025927
7001859
0.13893270193536106
7001860
0.030780380813213586
7001861
0.09258827758838432
7001862
0.1769572572844877
7001863
0.22314472081823442
7001864
0.2764259773421735
7001865
0.10949069705331944
7001866
0.21853735587751041
7001867
0.1099586969065493
7001868
0.20808438308051533
7001869
0.10441045527233199
7001870
0.13224157875151127
7001871
0.08806904497813949
7001872
0.11091263246618917
7001873
0.06894986992805094
7001874
0.12522706804966183
7001875
0.26745992071573915
7001876
0.22935084549326706
7001877
0.08869591320635464
7001878
0.029942440925068026
7001879
0.05670447213329401
7001880
0.23115858599208053
7001881
0.10393484707442169
7001882
0.0781335116482393
7001883
0.1929257155974993
7001884
0.13334157888034515
7001

0.08637385951618429
7002604
0.09419367059740194
7002605
0.12433511627295209
7002606
0.21657391019934208
7002607
0.1189424322167527
7002608
0.10576468244558042
7002609
0.15046243639756895
7002610
0.3058978065160271
7002611
0.14428762807262277
7002612
0.16189045730006169
7002613
0.08042785906920293
7002614
0.09954658853910928
7002615
0.2522869310932598
7002616
0.1831460900084467
7002617
0.1384742799826307
7002618
0.17748786800153313
7002619
0.1320731260412202
7002620
0.14153064302148816
7002621
0.19699183250908314
7002622
0.12358856504838601
7002623
0.12945144533146422
7002624
0.21058492041553187
7002625
0.0458991419360383
7002626
0.12001921920477708
7002627
0.38461987177088786
7002628
0.1964372611539025
7002629
0.07232724425823478
7002630
0.09534910350413843
7002631
0.18017979316392999
7002632
0.11480442579171707
7002633
0.10480736078838913
7002634
0.14612832082895877
7002635
0.26297911523570927
7002636
0.2102081266648986
7002637
0.265982491297157
7002638
0.12459806257437295
7002639
0.1

7003603
0.11635500746532453
7003604
0.17826656257650503
7003605
0.09312239934849043
7003606
0.21667447192363698
7003607
0.04820791386071132
7003608
0.1527615871757171
7003609
0.19073431866093127
7003610
0.04622901258982023
7003611
0.20754222650428195
7003612
0.16797568550924163
7003613
0.12113618298149031
7003614
0.05639155353066451
7003615
0.26275941289178967
7003616
0.27400562035306386
7003617
0.12474268404122081
7003618
0.16065695526275148
7003619
0.135186163090677
7003620
0.16240760706179463
7003621
0.10865363775340271
7003622
0.1640640182376787
7003623
0.16726342761883928
7003624
0.20076149237217072
7003625
0.1730184263478131
7003626
0.1725439386874876
7003627
0.14773732394141964
7003628
0.4045614402485077
7003629
0.09583541578944477
7003630
0.20467388036406295
7003631
0.11783074191484852
7003632
0.10822305429220694
7003633
0.04908804094981987
7003634
0.38103133542227335
7003635
0.22017897758389238
7003636
0.28884875127407905
7003637
0.0332892455449743
7003638
0.16538241485851704


...

7093906
0.13806205240645364
7093907
0.040112020596769146
7093908
0.24261387372994586
7093909
0.06283066300697111
7093910
0.19794222497047856
7093911
0.2549284160098828
7093912
0.1344251368448335
7093913
0.14020715560352898
7093914
0.23918877051914766
7093915
0.16803886105702517
7093916
0.10147129923901743
7093917
0.08415367516958651
7093918
0.13207609734821854
7093919
0.08546807217742232
7093920
0.043666583238252225
7093921
0.07706861901830499
7093922
0.21384050198477333
7093923
0.18182417116204672
7093924
0.2186202234780642
7093925
0.04093914837616257
7093926
0.09227433160964431
7093927
0.22875442766644447
7093928
0.23648515260783592
7093929
0.25135958197296165
7093930
0.1488610476275767
7093931
0.12012631810047651
7093932
0.12323836894879991
7093933
0.15839101164709496
7093934
0.25901742698283536
7093935
0.21641922539138173
7093936
0.25738277469847604
7093937
0.09672645176171085
7093938
0.1527944908112841
7093939
0.1733186236232449
7093940
0.1023527291141299
7093941
0.103724199470114

7094520
0.13485720112196734
7094521
0.1446648860974764
7094522
0.3261826661170878
7094523
0.11686729504469334
7094524
0.07573650746057578
7094525
0.10933596085271939
7094526
0.15955110509257578
7094527
0.1896969560532373
7094528
0.10774485498050496
7094529
0.18385430009824086
7094530
0.08825543695482806
7094531
0.08913176338028005
7094532
0.1622454945742324
7094533
0.2916455610529994
7094534
0.05417908933426469
7094535
0.5374303806138649
7094536
0.06916031284250562
7094537
0.1689592436853258
7094538
0.14914015801838632
7094539
0.21907306415968006
7094540
0.18987920958494398
7094541
0.06020026039759915
7094542
0.16666394309800797
7094543
0.07308174256491527
7094544
0.32377414855219905
7094545
0.2156352100950864
7094546
0.13130024003750382
7094547
0.1322683330034148
7094548
0.22700907047837196
7094549
0.1193823589283722
7094550
0.10334605603746694
7094551
0.2670636578247633
7094552
0.15318158282485664
7094553
0.0585458338194473
7094554
0.21199037697106532
7094555
0.26266797104600553
7094

0.09924079877997927
7095464
0.09141397862829652
7095465
0.6816087012452636
7095466
0.1940278687040314
7095467
0.17079813658529788
7095468
0.09802418800431749
7095469
0.111801610709008
7095470
0.1595704998675934
7095471
0.3222390352901615
7095472
0.06929008029659679
7095473
0.10219828119514837
7095474
0.12976564632408658
7095475
0.15768680296500107
7095476
0.08823275587472994
7095477
0.22617209772207855
7095478
0.07040947190723452
7095479
0.1383786820731845
7095480
0.11050876636450603
7095481
0.13335672276759103
7095482
0.24451714368273939
7095483
0.0717753067209716
7095484
0.18015646625202675
7095485
0.1672802949623485
7095486
0.10647311733505796
7095487
0.12767775558096808
7095488
0.09194919811505196
7095489
0.08097151196529642
7095490
0.07231621763025395
7095491
0.08607660408672733
7095492
0.24850398025644455
7095493
0.248857354331631
7095494
0.05705904791666342
7095495
0.2848911109483415
7095496
0.16819797984289878
7095497
0.28875632865330536
7095498
0.09689316183980577
7095499
0.34

0.19245828382151448
7096263
0.2564551110598632
7096264
0.2455600442797596
7096265
0.12706653354144112
7096266
0.2860345766851457
7096267
0.2831554237609821
7096268
0.156315110934469
7096269
0.06212243961945911
7096270
0.2265924598049462
7096271
0.23222450298568223
7096272
0.18779862122604865
7096273
0.24971653335831936
7096274
0.11232937377136526
7096275
0.16177930064553447
7096276
0.09353413033854835
7096277
0.09289503370094375
7096278
0.21562564892563965
7096279
0.183193342943819
7096280
0.1497738856051976
7096281
0.19906129060137145
7096282
0.22162366305979442
7096283
0.2246965072835638
7096284
0.09051870645653022
7096285
0.10803926699484812
7096286
0.5785657602242065
7096287
0.16083737923380287
7096288
0.058387361981275594
7096289
0.18900237497309966
7096290
0.27705950605037566
7096291
0.22201471610095244
7096292
0.08139061530851092
7096293
0.14014204694708732
7096294
0.13481605234720098
7096295
0.05033090995532908
7096296
0.15338035122060611
7096297
0.12237828970960066
7096298
0.0

0.2248338026176596
7097053
0.08512042997390495
7097054
0.12897637221966496
7097055
0.14099704152545475
7097056
0.3485441416623245
7097057
0.18001432879658552
7097058
0.1552597313893415
7097059
0.1046555094748963
7097060
0.06638433499615583
7097061
0.16395990576350092
7097062
0.2653273873311387
7097063
0.15622943558322896
7097064
0.19647494749186636
7097065
0.1888175774531966
7097066
0.12170155029734807
7097067
0.1428812680895494
7097068
0.18929702877339644
7097069
0.11779277328935948
7097070
0.07305675107502835
7097071
0.1698015921165776
7097072
0.11625728652646873
7097073
0.1476098819992305
7097074
0.208149195597898
7097075
0.15133114813111537
7097076
0.2629703105192896
7097077
0.06495459934126512
7097078
0.04013793380720128
7097079
0.1637498374077396
7097080
0.1580880760731648
7097081
0.10322681607909548
7097082
0.1440527132782132
7097083
0.26612901765905944
7097084
0.18094441459286956
7097085
0.16941887558226856
7097086
0.1303357790190679
7097087
0.14481797300774973
7097088
0.067004